Оцениваение: минимально в 2 раза больше обычного дз (если сделаете совсем много, возможны дополнительные бонусы).
На чуть больше половины баллов за проект достаточно правильно сделать CharCNN-BLSTM-CRF для NER и вариант модели Miwa & Bansal (2016) для RE.
Дедлайн 1 мая 9 утра (4 недели)

In [54]:
import brat_format
import os
from nltk.tokenize import sent_tokenize
import pickle 

import pandas as pd

In [55]:
ner_types = {"OUT", "ACT", "BIN", "CMP", "ECO", "INST", "MET", "SOC", "QUA"}
# rel_types = {"NNG", "NNT", "NPS", "FNG", "FNT", "FPS", "PNG", "PNT", "PPS", "GOL", "TSK"}

## Читаю данные

In [56]:
def read_data(ann_file):
    br_doc = brat_format.read_file(ann_file)
    segs = sent_tokenize(br_doc.txt_data, language="russian")

    count_chars = 0
    cnt_global_ners = 0
    ners_text = []
    for i in range(len(segs)):
        seg = segs[i]

        while br_doc.txt_data[count_chars] != seg[0]:
            count_chars += 1

        end_seg = count_chars + len(seg)
        ners_this_seg = []
        while br_doc.ners and br_doc.ners[0][1] <= end_seg:
            ent = br_doc.ners.pop(0)
            #print(ent)
            ners_this_seg.append((ent[0] - count_chars, ent[1] - count_chars, ent[2]))

        count_chars += len(seg)
        ners_text.append(ners_this_seg)

    #     print('-' * 20, 'NER', '-' * 20)
    #     for ner in ners_this_seg:
    #         print(ner[0], ner[1], seg[ner[0]: ner[1]], ner[2])

    #     print('\n\n\n')
    
    return segs, ners_text

## READ TRAIN_VAL

In [57]:
train_val_texts, train_val_ners = [], [] 

for i in range(1, 4):
    PATH = f'train_data/train_part_{i}/train_part_{i}/'
    texts = []
    ners = []

    list_files = os.listdir(PATH)
    ann_files = [x for x in list_files if x.endswith('ann')]
    
    for ann_file in ann_files:
        segs, ners_file = read_data(PATH + ann_file)
        
        texts += segs
        ners += ners_file
    
    train_val_texts += texts
    train_val_ners += ners
    
print(len(train_val_texts), len(train_val_ners))

10631 10631


In [58]:
i = 343

print(len(train_val_texts[i]), '\n\n', train_val_texts[i], '\n')

for ner in train_val_ners[i]:
    print(train_val_texts[i][ner[0]: ner[1]], ner[2])

295 

 Всего за 2015 год в районе:
 - введено   в эксплуатацию  - 18544 кв.м. площади, в т.ч. 
- 11090 кв. м. жилой, 
- выдано 153 разрешения на строительство, реконструкцию, капитальный ремонт объектов социальной сферы, жилых домов, газовых и электрических сетей,
- введено в эксплуатацию 50 объектов. 

введено   в эксплуатацию BIN
выдано BIN
разрешения на строительство, реконструкцию, капитальный ремонт объектов социальной сферы, жилых домов, газовых и электрических сетей ACT
введено в эксплуатацию BIN


## READ TEST

In [59]:
PATH = 'test_data/test_ner_only/'

list_files = os.listdir(PATH)
ann_files = [x for x in list_files if x.endswith('ann')]

test_texts = []
test_ners = []
for ann_file in ann_files:
    segs, ners_text = read_data(PATH + ann_file)

    test_texts += segs
    test_ners += ners_text
    
print(len(test_texts), len(test_ners))

19473 19473


In [60]:
train_val_ners_exist = set()

for spans in train_val_ners:
    for span in spans:
        train_val_ners_exist.add(span[2])
        
print(train_val_ners_exist)
print(ner_types - train_val_ners_exist)

{'CMP', 'ECO', 'ACT', 'BIN', 'QUA', 'MET', 'INST', 'SOC'}
{'OUT'}


In [61]:
test_ners_exist = set()

for spans in test_ners:
    for span in spans:
        test_ners_exist.add(span[2])
        
print(test_ners_exist)
print(ner_types - test_ners_exist)

{'ECO', 'CMP', 'ACT', 'BIN', 'QUA', 'MET', 'INST', 'SOC'}
{'OUT'}


## Labels to label_id

In [62]:
tag_2_tag_id = {v: k + 1 for k, v in enumerate(ner_types - {'OUT'})}
tag_2_tag_id['OUT'] = 0

tag_id_2_tag = {v: k for k, v in tag_2_tag_id.items()}

tag_2_tag_id

{'ECO': 1,
 'CMP': 2,
 'ACT': 3,
 'BIN': 4,
 'QUA': 5,
 'MET': 6,
 'INST': 7,
 'SOC': 8,
 'OUT': 0}

In [63]:
from enum import IntEnum
from collections import namedtuple

from razdel import tokenize

from ipymarkup import show_box_markup
from ipymarkup.palette import palette

class Label(IntEnum):
    OUTER = 0
    BEGIN = 1
    INNER = 2

Sample = namedtuple("Sample", "text,tokens,spans,labels")

def text_span_to_sample(text, spans):
    labels = []
    tokens = list(tokenize(text))
    
    spans_copy = spans.copy()
    for token in tokens:
        label = tag_2_tag_id['OUT']
        
        if spans_copy:
            span_begin, span_end, tag = spans_copy[0]

            if token.start == span_begin or (token.start > span_begin and token.stop <= span_end):
                label = tag_2_tag_id[tag]

            if token.stop == span_end:
                spans_copy.pop(0)
            
        labels.append(label)
    
    return Sample(text, tokens, spans, labels)

In [64]:
# Train
train_val = []
for text, spans in zip(train_val_texts, train_val_ners):
    train_val.append(text_span_to_sample(text, spans))

In [65]:
# Test
test = []
for text, spans in zip(test_texts, test_ners):
    test.append(text_span_to_sample(text, spans))

In [66]:
i = 9346

show_box_markup(train_val[i].text, train_val[i].spans)
print(train_val[i].labels)
print(len(train_val))

[0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
10631


In [67]:
i = 7

show_box_markup(test[i].text, test[i].spans)
print(test[i].labels)
print(len(test))

[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 6, 0, 6, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
19473


In [68]:
len(train_val)

10631

In [69]:
import random
random.shuffle(train_val)

train = train_val[:9500]
val = train_val[9500:]

In [70]:
char_set = ["<pad>", "<unk>"] + list({ch for sample in train for token in sample.tokens for ch in token.text})
print(char_set)

['<pad>', '<unk>', 'щ', 'p', '+', 'V', 'n', 'е', 'T', 'ы', 'ф', 'K', 'Ё', 'Д', 'G', '7', 'M', '│', 'о', '2', 'Ж', '3', 'н', '8', ':', '5', 'ъ', 'у', '«', ']', 'Ч', 'j', 'и', '4', 'Т', 'z', 'м', 'Ы', 'u', 'ш', '\uf0b7', '■', 'f', 'Э', 'S', '-', '{', '┌', 'Е', 'Л', 'g', 'Я', 'Р', 'Н', 'с', 'i', 'Ш', '»', '0', 'Ъ', 'F', '•', 'з', 'А', 'Ц', 'v', 'H', 'л', '}', 'п', 'd', '┬', '!', '>', 'В', 'x', 'y', 's', 'г', 'X', '‰', 'Σ', 'ә', 'к', '.', '─', 'в', '└', 'm', 'С', ',', 'ь', '=', 'e', 'Б', 'ж', 'L', 'э', 'К', 'б', '”', '×', '–', 'О', 'P', '%', 'k', 'R', 'o', 'җ', '∑', 'ё', 't', '…', '"', ';', '@', 'х', '├', '6', 'Й', 'h', '№', 'З', 'b', 'р', '/', 'а', 'N', 'I', ')', 'Щ', 'П', 'Ю', 'A', 'М', 'Ә', '1', 'C', 'c', 'U', 'a', 'ч', 'Ö', 'т', '(', 'w', 'ю', '<', 'я', 'l', 'У', '[', 'O', '┘', '—', '┼', '“', '9', 'Х', 'D', '┤', 'д', 'Z', 'E', 'И', 'І', '┐', 'r', 'Г', '_', '£', 'Ф', 'й', 'Ь', 'ц', '*', 'B']


Для каждого слова сохраняем его символьный состав, а в остальном старый добрый пайплайн

In [71]:
stat = []

for i in range(len(train_val)):
    stat.append(len(train_val[i].text))

pd.Series(stat).describe(percentiles=[x/10 for x in range(10)])

count    10631.000000
mean       231.197065
std        286.775179
min          1.000000
0%           1.000000
10%         10.000000
20%         73.000000
30%        105.000000
40%        134.000000
50%        166.000000
60%        202.000000
70%        248.000000
80%        320.000000
90%        461.000000
max       7562.000000
dtype: float64

In [77]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, RandomSampler

class NerCharsDataset(Dataset):
    def __init__(self, samples, char_set, max_seq_len=500, max_char_seq_len=50):
        assert len(samples) != 0
        self.samples = []
        self.tokens = []
        self.texts = []
        for sample in samples:
            inputs = torch.zeros((max_seq_len, max_char_seq_len), dtype=torch.long)
            for token_num, token in enumerate(sample.tokens[:max_seq_len]):
                for ch_num, ch in enumerate(token.text[:max_char_seq_len]):
                    char_index = char_set.index(ch) if ch in char_set else char_set.index("<unk>")
                    inputs[token_num][ch_num] = char_index
            labels = torch.zeros((max_seq_len,), dtype=torch.long)
            input_labels = [int(i) for i in sample.labels[:max_seq_len]]
            labels[:len(input_labels)] = torch.LongTensor(input_labels)
            self.samples.append((torch.LongTensor(inputs), torch.LongTensor(labels)))
            self.tokens.append(sample.tokens[:max_seq_len])
            self.texts.append(sample.text)
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        return self.samples[index]

    def get_tokens(self, index):
        return self.tokens[index]
    
    def get_text(self, index):
        return self.texts[index]


BATCH_SIZE = 64

train_data = NerCharsDataset(train, char_set)
train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, sampler=train_sampler)

val_data = NerCharsDataset(val, char_set)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)

test_data = NerCharsDataset(test, char_set)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [78]:
for sample in train_loader:
    inputs, labels = sample
    print(inputs.size())
    print(labels.size())
    break

# inputs: batch_size x num_words x num_chars
# labels: batch_size x num_words

torch.Size([64, 500, 50])
torch.Size([64, 500])


In [79]:
weights_classes = torch.Tensor([0.3] + [1.0 for _ in range(8)])

In [84]:
import torch
import torch.nn as nn
from pytorch_lightning import LightningModule
from pytorch_lightning.metrics import Accuracy

from torchmetrics import F1Score
from TorchCRF import CRF

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping
 

class TemplateModel(LightningModule):
    def __init__(self, classes_count=9):
        super().__init__()
        
        self.loss = nn.CrossEntropyLoss()
        self.valid_fmera = F1Score(task="multiclass", num_classes=9, average='macro', mdmc_average='global')
        self.test_fmera = F1Score(task="multiclass", num_classes=9, average='macro', mdmc_average='global')
#         self.valid_accuracy = Accuracy()
#         self.test_accuracy = Accuracy()
    
    def forward(self, inputs, labels):
        raise NotImplementedError("forward not implemented")
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-2)
        return [optimizer]
    
    def training_step(self, batch, _):
        inputs, labels = batch
        loss, logits = self(inputs, labels)
        return loss
    
    def validation_step(self, batch, _):
        inputs, labels = batch
        val_loss, logits = self(inputs, labels)
        self.log("val_loss", val_loss, prog_bar=True)
        
        predicted_labels = logits.argmax(dim=2)
        vf = self.valid_fmera(predicted_labels, labels)
        self.log("val_fmera", vf.item(), prog_bar=True, on_step=True, on_epoch=True)
        
#         self.valid_accuracy.update(logits, labels)
#         self.log("val_acc", self.valid_accuracy)

    def validation_epoch_end(self, outs):
        print('')
        #self.log("val_fmera_epoch", self.valid_fmera.compute(), prog_bar=True)
        #self.log("val_acc_epoch", self.valid_accuracy.compute(), prog_bar=True)

    def test_step(self, batch, _):
        inputs, labels = batch
        test_loss, logits = self(inputs, labels)
        self.log("test_loss", test_loss, prog_bar=True)
        
        predicted_labels = logits.argmax(dim=2)
        tf = self.test_fmera(predicted_labels, labels)
        self.log("test_fmera", tf.item(), prog_bar=True, on_step=True, on_epoch=True)
        
#         self.test_accuracy.update(logits, labels)
#         self.log("test_acc", self.test_accuracy)

    def test_epoch_end(self, outs):
        print('')
        self.log("test_fmera_epoch", self.test_fmera.compute(), prog_bar=True)
        #self.log("test_acc_epoch", self.test_accuracy.compute(), prog_bar=True)

In [85]:
class CharFFLstmModel(TemplateModel):
    def __init__(self, char_set_size, weights_classes, char_embedding_dim=8, classes_count=9,
                 word_embedding_dim=32, lstm_embedding_dim=32, char_max_seq_len=50):
        super().__init__()
        
        self.embeddings_layer = nn.Embedding(char_set_size, char_embedding_dim)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(char_embedding_dim * char_max_seq_len, word_embedding_dim)
        self.relu = nn.ReLU()
        self.lstm_layer = nn.LSTM(word_embedding_dim, lstm_embedding_dim // 2, batch_first=True, bidirectional=True)
        self.out_layer = nn.Linear(lstm_embedding_dim, classes_count)
        #self.loss = nn.CrossEntropyLoss(weight=weights_classes)
        self.loss = CRF(classes_count, batch_first=True)
        #self.softmax = torch.nn.Softmax(dim=2)

    def forward(self, inputs, labels):
        projections = self.embeddings_layer.forward(inputs)
        projections = projections.reshape(projections.size(0), projections.size(1), -1)
        projections = self.relu(self.linear(projections))
        projections = self.dropout(projections)
        output, _= self.lstm_layer(projections)
        output = self.dropout(output)
        logits = self.out_layer.forward(output)
        #logits = self.softmax(logits)
        #logits = logits.transpose(1, 2)
        loss = -self.loss(logits, labels)
        
        return loss, logits
    
char_ff_lstm_model = CharFFLstmModel(len(char_set), weights_classes, classes_count=len(ner_types))

In [86]:
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0,
    patience=8,
    verbose=True,
    mode="min" 
)

trainer = Trainer(
    gpus=1,
    checkpoint_callback=False,
    accumulate_grad_batches=1,
    max_epochs=200,
    progress_bar_refresh_rate=10,
    callbacks=[early_stop_callback])
trainer.fit(char_ff_lstm_model, train_loader, val_loader)
trainer.test(char_ff_lstm_model, test_loader)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name             | Type      | Params
-----------------------------------------------
0 | loss             | CRF       | 99    
1 | valid_fmera      | F1Score   | 0     
2 | test_fmera       | F1Score   | 0     
3 | embeddings_layer | Embedding | 1.4 K 
4 | dropout          | Dropout   | 0     
5 | linear           | Linear    | 12.8 K
6 | relu             | ReLU      | 0     
7 | lstm_layer       | LSTM      | 6.4 K 
8 | out_layer        | Linear    | 297   
-----------------------------------------------
21.1 K    Trainable params
0         Non-trainable params
21.1 K    Total params
0.084     Total estimated model params size (MB)


Epoch 0:  90%|████▍| 150/167 [01:32<00:10,  1.63it/s, loss=4.18e+03, v_num=40, val_loss=6.55e+4, val_fmera_epoch=0.110]
Validating: 0it [00:00, ?it/s]
Validating: 100%|██████████████████████████████████████████████████████████████████████| 18/18 [00:03<00:00,  4.90it/s]
Epoch 0: 100%|█| 167/167 [01:35<00:00,  1.74it/s, loss=3.95e+03, v_num=40, val_loss=3.84e+3, val_fmera_epoch=0.110, val
Epoch 1:  90%|▉| 150/167 [01:30<00:10,  1.66it/s, loss=1.93e+03, v_num=40, val_loss=3.84e+3, val_fmera_epoch=0.110, val
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                                                               | 0/18 [00:00<?, ?it/s]
Epoch 1:  96%|▉| 160/167 [01:32<00:04,  1.73it/s, loss=1.93e+03, v_num=40, val_loss=3.84e+3, val_fmera_epoch=0.110, val
Validating: 100%|██████████████████████████████████████████████████████████████████████| 18/18 [00:03<00:00,  4.97it/s]
Epoch 1: 100%|█| 167/167 [01:34<00:00,  1.77it/s, loss=1.86e+03, v_num=40, val_loss=1.73e+3, val_f

Epoch 12:  96%|▉| 160/167 [01:32<00:04,  1.73it/s, loss=1.23e+03, v_num=40, val_loss=1.06e+3, val_fmera_epoch=0.284, va
Validating: 100%|██████████████████████████████████████████████████████████████████████| 18/18 [00:03<00:00,  4.98it/s]
Epoch 12: 100%|█| 167/167 [01:34<00:00,  1.77it/s, loss=1.13e+03, v_num=40, val_loss=1.05e+3, val_fmera_epoch=0.317, va
Epoch 13:  90%|▉| 150/167 [01:31<00:10,  1.65it/s, loss=1.15e+03, v_num=40, val_loss=1.05e+3, val_fmera_epoch=0.317, va
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                                                               | 0/18 [00:00<?, ?it/s]
Epoch 13:  96%|▉| 160/167 [01:33<00:04,  1.72it/s, loss=1.15e+03, v_num=40, val_loss=1.05e+3, val_fmera_epoch=0.317, va
Validating: 100%|██████████████████████████████████████████████████████████████████████| 18/18 [00:03<00:00,  4.94it/s]
Epoch 13: 100%|█| 167/167 [01:34<00:00,  1.76it/s, loss=1.17e+03, v_num=40, val_loss=1.03e+3, val_fmera_epoch=0.331, va
Epoch 14:

Epoch 25:  90%|▉| 150/167 [01:34<00:10,  1.58it/s, loss=1.13e+03, v_num=40, val_loss=932.0, val_fmera_epoch=0.388, val_
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                                                               | 0/18 [00:00<?, ?it/s]
Epoch 25:  96%|▉| 160/167 [01:37<00:04,  1.65it/s, loss=1.13e+03, v_num=40, val_loss=932.0, val_fmera_epoch=0.388, val_
Validating: 100%|██████████████████████████████████████████████████████████████████████| 18/18 [00:03<00:00,  4.60it/s]
Epoch 25: 100%|█| 167/167 [01:38<00:00,  1.69it/s, loss=1.07e+03, v_num=40, val_loss=920.0, val_fmera_epoch=0.391, val_
Epoch 26:  90%|▉| 150/167 [01:32<00:10,  1.62it/s, loss=1.04e+03, v_num=40, val_loss=920.0, val_fmera_epoch=0.391, val_
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                                                               | 0/18 [00:00<?, ?it/s]
Epoch 26:  96%|▉| 160/167 [01:34<00:04,  1.69it/s, loss=1.04e+03, v_num=40, val_loss=920.0, val_fm

ModuleAttributeError: 'CharFFLstmModel' object has no attribute 'test_accuracy'

In [87]:
torch.save(char_ff_lstm_model.state_dict(), 'model_3004_1.pth')

In [37]:
! du -hs model_2704.pth

692K	model_2704.pth


In [141]:
# char_ff_lstm_model = CharFFLstmModel(len(char_set), weights_classes, classes_count=len(ner_types))
# char_ff_lstm_model.load_state_dict(torch.load(filepath))

# Оценка качества

In [89]:
MAX_SEQ_LEN = 500

test_sizz = len(test)

In [91]:
model = char_ff_lstm_model

total_tp = 0
total_fp = 0
total_fn = 0

global_true = []
global_pred = []
labels_pred = []

for batch_index, batch in enumerate(test_loader):
    #print(batch_index)
    inputs, true_labels = batch
    batch_size = inputs.size(0)
    _, logits = model(inputs.to("cuda"), true_labels.to("cuda"))
    
    predicted_labels = logits.argmax(dim=2).detach().cpu()
    labels_pred.append(predicted_labels)
    
    for i in range(BATCH_SIZE):
        num = batch_index * BATCH_SIZE + i
        if num == test_sizz:
            break
        
        #print('text: ', i)
        ners_model = []
        
        tokens = list(tokenize(test_texts[num]))
        tokens = tokens[:MAX_SEQ_LEN]

        indices = [i for i, x in enumerate(predicted_labels[i]) if x != 0]
        indices_tensor = torch.Tensor(indices).type(torch.int64)

        vals = predicted_labels[i].gather(dim=0, index=indices_tensor)
        tokens_selected = [tokens[i] for i in indices]

        pred_tag_id = None
        start = None
        stop = None
        for token, val in zip(tokens_selected, vals):
            tag_id = int(val.item())
            
            if tag_id == pred_tag_id and token.start == stop + 1:
                stop = token.stop
            else:
                if pred_tag_id is not None:
                    ners_model.append((start, stop, tag_id_2_tag[pred_tag_id]))
                    
                pred_tag_id = tag_id
                start = token.start
                stop = token.stop
                
        if (len(tokens_selected) != 0 and
            (len(ners_model) == 0 or ners_model[-1][1] != stop)):
            ners_model.append((start, stop, tag_id_2_tag[pred_tag_id]))
            
        global_true.append(test_ners[i])
        global_pred.append(ners_model)

        tp, fp, fn = evaluate_ners.cacl_ner_tp_fp_fn(test_ners[i], ners_model)
        total_tp += tp
        total_fp += fp
        total_fn += fn
        
    if batch_index % 100 == 0:
        print('batch_index: ', batch_index)
        
#     if batch_index == 1:
#         break
        
precision, recall = evaluate_ners.compute_precision_and_recall(total_tp, total_fp, total_fn)
f_measure = 2 * precision * recall / (precision + recall)

batch_index:  0
batch_index:  100
batch_index:  200
batch_index:  300


In [92]:
total_tp, total_fp, total_fn

(64, 152551, 61686)

In [93]:
precision, recall = evaluate_ners.compute_precision_and_recall(total_tp, total_fp, total_fn)
f_measure = 2 * precision * recall / (precision + recall)

print(precision, recall, f_measure)

0.0010364372469635627 0.00041935589555417227 0.0005971124017446879
